In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

# Speed-Prediction

In [2]:
df = pd.read_csv("outlier_removed_org.csv")
df.sample(5)

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,pm,stator_yoke,ambient,torque
822583,68.716985,49.809901,58.406562,-37.289124,56.175167,1499.980225,-15.494592,70.794579,58.300533,53.073852,25.457240,50.000000
493564,20.755384,44.599976,113.316002,-30.253600,91.738765,500.018157,-79.940579,188.410821,75.109815,72.344401,24.152944,150.000007
433954,69.270348,19.449896,66.068680,-111.728729,52.814259,2999.963379,-124.302979,113.205444,57.926064,39.451782,23.443586,99.973587
375095,73.561066,18.636902,23.990009,-0.363043,24.142956,1499.986572,-2.000791,1.096605,33.154404,22.413567,23.372068,0.069424
484166,1.895813,68.244739,96.774651,0.713429,85.238939,13.665952,-2.057795,1.138674,71.920140,75.846451,24.530464,0.039133


In [3]:
X = df.drop(columns=['motor_speed'])
X.sample(5)

,u_q,coolant,stator_winding,u_d,stator_tooth,i_d,i_q,pm,stator_yoke,ambient,torque
173337,131.743118,23.220242,36.483273,-3.020050,36.039875,-45.508091,1.171260,44.946644,31.360949,23.510765,-5.305682e-01
294382,119.381119,18.993454,63.932823,-52.012161,52.917957,-118.984337,22.732635,65.723526,38.773849,22.395842,2.001521e+01
744397,46.650700,67.991580,104.791162,-118.543994,91.677664,-158.064510,115.707295,85.847361,79.344978,26.384478,1.069333e+02
415816,59.020966,65.024521,104.791176,-115.770660,89.096138,-165.044769,63.193829,75.732971,75.762482,26.427488,5.825471e+01
692091,-2.036200,50.128496,58.371314,1.198199,55.835528,-2.000682,1.097439,64.082791,53.154303,25.186641,-1.935200e-69


In [4]:
y = df[['motor_speed']]
y.sample(5)

,motor_speed
810616,999.986989
413392,149.923111
72885,4499.958008
87138,99.985130
403983,0.000804


In [5]:
# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Step 2: Create an XGBoost model
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Specify the objective for regression tasks
    learning_rate=0.8,             # Step size shrinkage used in updates to prevent overfitting
    n_estimators=100,              # Number of boosting rounds (trees)
    max_depth=6,                   # Maximum depth of each tree
    subsample=0.8,                 # Fraction of samples used for fitting each tree
    colsample_bytree=0.8,          # Fraction of features used for fitting each tree
    random_state=42                # For reproducibility
)

In [7]:
# Step 3: Fit the model to the training data
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.8, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [8]:
# Step 4: Make predictions on the test data
y_pred = model.predict(X_test)

In [9]:
#Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 2508.3321041150875
R-squared: 0.9992153709212537


In [ ]:

# # Define a range of random state values and model parameters to iterate over
# random_states = [42, 123, 456]
# learning_rates = [0.1, 0.01]
# n_estimators_list = [100, 200, 300]
# max_depths = [3, 5]

# best_accuracy = 0
# best_random_state = None
# best_params = {}

# # Loop through random state and model parameter combinations
# for random_state in random_states:
#     for learning_rate in learning_rates:
#         for n_estimators in n_estimators_list:
#             for max_depth in max_depths:
#                 # Split the data into training and testing sets using the current random state
#                 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

#                 # Create the XGBoost model with the current parameter combination
#                 model = xgb.XGBRegressor(
#                     objective='reg:squarederror',
#                     learning_rate=learning_rate,
#                     n_estimators=n_estimators,
#                     max_depth=max_depth,
#                     random_state=random_state
#                 )

#                 # Perform cross-validation and calculate mean accuracy
#                 cv_scores = cross_val_score(model, X_train, y_train, cv=5)
#                 mean_accuracy = np.mean(cv_scores)

#                 # Check if the current accuracy is better than the best accuracy found so far
#                 if mean_accuracy > best_accuracy:
#                     best_accuracy = mean_accuracy
#                     best_random_state = random_state
#                     best_params = {
#                         'learning_rate': learning_rate,
#                         'n_estimators': n_estimators,
#                         'max_depth': max_depth
#                     }

# # Print the best accuracy and corresponding random state and model parameters
# print("Best Accuracy:", best_accuracy)
# print("Best Random State:", best_random_state)
# print("Best Model Parameters:", best_params)

# Torque-Prediction

In [51]:
X = df.drop(columns=['torque'])
y = df[['torque']]

In [52]:
# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
# Step 2: Create an XGBoost model
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Specify the objective for regression tasks
    learning_rate=1,             # Step size shrinkage used in updates to prevent overfitting
    n_estimators=100,              # Number of boosting rounds (trees)
    max_depth=12,                   # Maximum depth of each tree
    subsample=0.8,                 # Fraction of samples used for fitting each tree
    colsample_bytree=0.8,          # Fraction of features used for fitting each tree
    random_state=42                # For reproducibility
)

In [58]:
# Step 3: Fit the model to the training data
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [59]:
# Step 4: Make predictions on the test data
y_pred = model.predict(X_test)

In [60]:
#Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.3445811227189737
R-squared: 0.9998921549379178


# Coolant-Prediction

In [3]:
X = df.drop(columns=['coolant'])
y = df[['coolant']]

In [4]:
X.head()

,u_q,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,pm,stator_yoke,ambient,torque
0,90.301086,46.627861,-92.912933,37.219505,4999.959473,-141.435104,52.935848,35.045197,27.430012,20.899378,48.561615
1,90.450218,46.660896,-92.856468,37.326965,4999.958008,-141.503265,52.955212,35.090080,27.474701,21.009548,48.472355
2,90.530746,46.493481,-92.806671,37.351593,4999.955078,-141.524933,52.981113,35.137783,27.520298,21.024719,48.556767
3,90.633858,46.390057,-92.744507,37.405773,4999.953613,-141.659454,52.957489,35.175957,27.596558,21.067995,48.486523
4,90.783813,46.461845,-92.682083,37.428577,4999.952637,-141.654755,52.955505,35.237061,27.663696,21.140137,48.430492


In [5]:
y.head()

,coolant
0,19.310339
1,19.312284
2,19.303518
3,19.265827
4,19.215105


In [6]:
# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Step 2: Create an XGBoost model
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Specify the objective for regression tasks
    learning_rate=1,             # Step size shrinkage used in updates to prevent overfitting
    n_estimators=100,              # Number of boosting rounds (trees)
    max_depth=15,                   # Maximum depth of each tree
    subsample=0.8,                 # Fraction of samples used for fitting each tree
    colsample_bytree=0.8,          # Fraction of features used for fitting each tree
    random_state=42                # For reproducibility
)

In [8]:
# Step 3: Fit the model to the training data
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [9]:
# Step 4: Make predictions on the test data
y_pred = model.predict(X_test)

In [10]:
#Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 1.2401710078839474
R-squared: 0.9956127095837013


In [11]:
import joblib

In [12]:
filename = 'coolant_pred.joblib'
joblib.dump(model, filename)

['coolant_pred.joblib']